In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

100%|███████████████████████████████████████████████████████████████████████████| 39291/39291 [00:48<00:00, 810.27it/s]


In [37]:
df["last_lecture"] = df.groupby("user_id")["content_id"].shift(1)

df_all = []
for _, w_df in tqdm.tqdm(df.groupby("user_id")):
    w_df["last_lecture"] = [x[0] if x[1] == 1 else np.nan for x in w_df[["last_lecture", "content_type_id"]].values]
    w_df["last_lecture"] = w_df["last_lecture"].fillna(method="ffill").fillna(-1)
    df_all.append(w_df)

df = pd.concat(df_all)
df["from_last_lecture"] = df.groupby(["user_id", "last_lecture"]).transform("cumcount")
df["from_last_lecture"] = [x[0] if not x[1]==-1 else np.nan for x in df[["from_last_lecture", "last_lecture"]].values]
df["from_last_lecture"] = df["from_last_lecture"].fillna(-1)
df.groupby("from_last_lecture")["answered_correctly"].describe().head(40)

In [38]:
df["from_last_lecture"].fillna(-1).value_counts().head(20)

-1.0     2330000
 0.0      193617
 1.0      187734
 2.0      185810
 3.0      163785
 4.0      157661
 5.0      151175
 6.0      146635
 7.0      140917
 8.0      136429
 9.0      132510
 10.0     128965
 11.0     125720
 12.0     122741
 13.0     116051
 14.0     113120
 15.0     110151
 16.0     103758
 17.0      99277
 18.0      96788
Name: from_last_lecture, dtype: int64

,count,mean,std,min,25%,50%,75%,max
from_last_lecture,,,,,,,,
-1.0,2329986.0,0.594969,0.490898,0.0,0.0,1.0,1.0,1.0
0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,187704.0,0.727891,0.445047,0.0,0.0,1.0,1.0,1.0
2.0,185721.0,0.734893,0.441392,0.0,0.0,1.0,1.0,1.0
3.0,162783.0,0.648477,0.477447,0.0,0.0,1.0,1.0,1.0
4.0,157547.0,0.643700,0.478907,0.0,0.0,1.0,1.0,1.0
5.0,151061.0,0.639887,0.480034,0.0,0.0,1.0,1.0,1.0
6.0,146581.0,0.640049,0.479987,0.0,0.0,1.0,1.0,1.0
7.0,140864.0,0.641136,0.479669,0.0,0.0,1.0,1.0,1.0
